In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from numpy import cos
from math import log
from scipy.interpolate import interp1d
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize

#### As equações do modelo:

$dShdt = -a_{curr} . b2 . (I_M / N) . S_H \\
dIhdt = a_{curr} . b2 . (I_M / N) . S_H - \gamma . I_H \\
dRhdt = \gamma . I_H$

$dSmdt = b_{curr} - a_{curr} . b1 . (I_H / N) . S_M - \mu_{curr} . S_M \\
dEmdt = a_{curr} . b1 . (I_H / N) . S_M - \mu_{curr} . E_M - b3_{curr} . E_M . l_{curr} \\
dImdt = b3_{curr} . E_M . l_{curr} - \mu_{curr} . I_M$

In [2]:
%display typeset

### Analisando o SIR:

In [3]:
var('S_H I_H a b2 I_M N gamma')

(S_H, I_H, a, b2, I_M, N, gamma)

In [4]:
dShdt = -a * b2 * (I_M / N) * S_H
dIhdt = a * b2 * (I_M / N) * S_H - gamma * I_H
dRhdt = gamma * I_H

Dado que o compartimento dos Recuperados é desacoplado da dinâmica, podemos ignorá-lo na análise.

#### Calculando o equilíbrio do SIR

In [5]:
solve([dShdt,dIhdt],[S_H,I_H])

[[S_H == 0, I_H == 0]]

#### Calculando a Jacobiana do SIR

In [6]:
jack_sir=jacobian([dShdt,dIhdt],[S_H,I_H])
jack_sir

[-I_M*a*b2/N           0]
[ I_M*a*b2/N      -gamma]

In [7]:
cp_sir = jack_sir.characteristic_polynomial()
cp_sir

x^2 + (I_M*a*b2/N + gamma)*x + I_M*a*b2*gamma/N

In [8]:
jack_sir.eigenvalues()

[-I_M*a*b2/N, -gamma]

### Analisando o SEI:

In [9]:
var('S_M E_M I_M b a b1 I_H N mu b3 l')

(S_M, E_M, I_M, b, a, b1, I_H, N, mu, b3, l)

In [10]:
dSmdt = b - a * b1 * (I_H / N) * S_M - mu * S_M
dEmdt = a * b1 * (I_H / N) * S_M - mu * E_M - b3 * E_M * l
dImdt = b3 * E_M * l - mu * I_M

#### Calculando o equilíbrio do SEI

In [11]:
solve([dSmdt,dEmdt,dIhdt],[S_M,E_M,I_M])

[[S_M == N*b/(I_H*a*b1 + N*mu), E_M == I_H*a*b*b1/(I_H*a*b1*b3*l + N*mu^2 + (I_H*a*b1 + N*b3*l)*mu), I_M == I_H*N*gamma/(S_H*a*b2)]]

#### Calculando a Jacobiana do SEI

In [12]:
jack_sei=jacobian([dSmdt,dEmdt,dIhdt],[S_M,E_M,I_M])
jack_sei

[-I_H*a*b1/N - mu                0                0]
[      I_H*a*b1/N       -b3*l - mu                0]
[               0                0       S_H*a*b2/N]

In [13]:
cp_sei = jack_sei.characteristic_polynomial()
cp_sei

x^3 + (I_H*a*b1/N - S_H*a*b2/N + b3*l + 2*mu)*x^2 + (-I_H*S_H*a^2*b1*b2/N^2 + I_H*a*b1*b3*l/N - S_H*a*b2*b3*l/N + I_H*a*b1*mu/N - 2*S_H*a*b2*mu/N + b3*l*mu + mu^2)*x - I_H*S_H*a^2*b1*b2*b3*l/N^2 - I_H*S_H*a^2*b1*b2*mu/N^2 - S_H*a*b2*b3*l*mu/N - S_H*a*b2*mu^2/N

In [14]:
jack_sei.eigenvalues()

[-(I_H*a*b1 + N*mu)/N, S_H*a*b2/N, -b3*l - mu]